In [1]:
%run /Common/config_sandbox

In [2]:
yourSandboxName = 'JamesA'

### list directory contents

In [4]:
display(dbutils.fs.ls("/mnt/SandboxStorage/SampleDataSets" ))

path,name,size
dbfs:/mnt/SandboxStorage/SampleDataSets/All/,All/,0
dbfs:/mnt/SandboxStorage/SampleDataSets/ByCompany/,ByCompany/,0
dbfs:/mnt/SandboxStorage/SampleDataSets/RandomSample/,RandomSample/,0
dbfs:/mnt/SandboxStorage/SampleDataSets/Snapshot/,Snapshot/,0


#Retrive the sample data set(small subset of trips)

In [6]:
testDF = spark.read.format('delta').load("/mnt/SandboxStorage/SampleDataSets/TripsData_Sampled_2019")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o471.load.
: org.apache.spark.sql.AnalysisException: /mnt/SandboxStorage/SampleDataSets/TripsData_Sampled_2019 doesn't exist;
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.pathNotExistsException(DeltaErrors.scala:262)
	at com.databricks.sql.transaction.tahoe.sources.DeltaDataSource$$anonfun$11.apply(DeltaDataSource.scala:159)
	at com.databricks.sql.transaction.tahoe.sources.DeltaDataSource$$anonfun$11.apply(DeltaDataSource.scala:156)
	at scala.Option.getOrElse(Option.scala:121)
	at com.databricks.sql.transaction.tahoe.sources.DeltaDataSource.createRelation(DeltaDataSource.scala:156)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:347)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:307)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:293)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:214)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 AnalysisException Traceback (most recent call last)
 <command-3982846140649653> in <module> 
 ----> 1 testDF = spark . read . format ( 'delta' ) . load ( "/mnt/SandboxStorage/SampleDataSets/TripsData_Sampled_2019" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in load (self, path, format, schema, **options) 
 164 self . options ( ** options ) 
 165 if isinstance ( path , basestring ) : 
 --> 166 return self . _df ( self . _jreader . load ( path ) ) 
 167 elif path is not None : 
 168 if type ( path ) != list : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 67 e.java_exception.getStackTrace()))
 68 if s . startswith ( 'org.apache.spark.sql.AnalysisException: ' ) : 
 ---> 69 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 70 if s . startswith ( 'org.apache.spark.sql.catalyst.analysis' ) : 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 AnalysisException : "/mnt/SandboxStorage/SampleDataSets/TripsData_Sampled_2019 doesn't exist;"

In [7]:
testDF.count()

#Explore the sample data

1. item 1
1. item 2

In [10]:
testDF.rdd.getNumPartitions()

### what type of data is in this dataset(largest value, smallest, ect)

In [12]:
display(testDF.describe())

### Get random sample of  rows

In [14]:
sampledDF = testDF.sample(.01)

display(sampledDF.groupby('CompanyId').count())

In [15]:
sampledDF.count()

### Write it back out to my storage

### Add columns for Year, Month and Day out to different columns(using start trip time, used later)

### Write the sample data set back out to disk in a different folder. Partitioned(multiple folders) by Year, month and day

> IE
>
> YYYY
>
> --MM
>
> ----DD

In [19]:
#by Year/Month/Day
sampledDF.write.partitionBy("year", 'month', 'day').format('delta').mode('overwrite').save('/mnt/SandboxStorage/%s/SmallSampledTripsByYear/' % yourSandboxName)#JamesA

#one file
sampledDF.coalesce(1).write.format('delta').mode('overwrite').save('/mnt/SandboxStorage/%s/SmallSampledTrips/' % yourSandboxName)

### Read back that dataset just written

In [21]:
retrivedDFByYear = spark.read.format('delta').load('/mnt/SandboxStorage/%s/SmallSampledTripsByYear/' % yourSandboxName)
retrivedDF = spark.read.format('delta').load('/mnt/SandboxStorage/%s/SmallSampledTrips/' % yourSandboxName)


In [22]:
print(retrivedDFByYear.count())
print(retrivedDFByYear.rdd.getNumPartitions())

In [23]:
print(retrivedDF.count())
print(retrivedDF.rdd.getNumPartitions())

In [24]:
display(retrivedDF)